<a href="https://colab.research.google.com/github/ASADULLAH999/LangChain-Hello-world-/blob/main/RAG_project_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install -qU langchain-pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 3.4 MB/s eta 0:00:00


In [3]:
!pip -q install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 8.6 MB/s eta 0:00:00


In [4]:

from langchain.schema import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]


In [5]:
from docx import Document

# Create a new Word document
doc = Document()

# Add a title
doc.add_heading('Pet Information', 0)

# Loop through the documents and add their content to the Word file
for document in documents:
    doc.add_heading(document.metadata["source"], level=1)
    doc.add_paragraph(document.page_content)

# Save the Word document
doc.save("/content/pet_information.docx")

In [6]:
print(documents)

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'), Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'), Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'), Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'), Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]


In [7]:
!pip install -Uq langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.8 MB/s eta 0:00:00


In [9]:
from google.colab import userdata

from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004",
                                          task_type="retrieval_document",
                                          google_api_key=userdata.get('GOOGLE_API_KEY'))
# embeddings.embed_query("What's our Q1 revenue?")

In [10]:

embeddings.embed_query("What's our Q1 revenue?")

[0.022226594388484955,
 -0.005298603791743517,
 -0.03638031706213951,
 -0.022343235090374947,
 0.021077357232570648,
 0.026357078924775124,
 -0.005140196997672319,
 -0.0047151208855211735,
 -0.020215705037117004,
 0.004603210836648941,
 0.021817319095134735,
 0.009138683788478374,
 0.10358211398124695,
 -0.005005174316465855,
 0.01096127089112997,
 -0.01904454454779625,
 0.005782195366919041,
 -0.015201744623482227,
 -0.13191628456115723,
 -0.0029817058239132166,
 0.01769891194999218,
 -0.010750621557235718,
 0.0003200707142241299,
 -0.0002848370058927685,
 -0.029578616842627525,
 -0.031128263100981712,
 -0.02592356875538826,
 -0.0026727376971393824,
 -0.014347424730658531,
 -0.013856600970029831,
 0.04075340926647186,
 0.0566372312605381,
 -0.025405442342162132,
 -0.07057256251573563,
 -0.013354412280023098,
 0.01110386848449707,
 -0.017656734213232994,
 0.049004241824150085,
 0.05370621383190155,
 -0.06309186667203903,
 -0.04680578410625458,
 0.04269225150346756,
 -0.0376492477953434

In [11]:

len(embeddings.embed_query("What's our Q1 revenue?")) # So, you can see that given dimension of prompt is 768

768

In [13]:
import os
from google.colab import userdata

from pinecone import Pinecone, ServerlessSpec

os.environ["PINECONE_API_KEY"] = userdata.get("PINECONE_API_KEY")

pinecone_api_key = os.environ["PINECONE_API_KEY"]

pc = Pinecone(api_key=pinecone_api_key)

In [14]:
import time
from pinecone import Pinecone, ServerlessSpec

index_name = "rag2-piaic-asadullah"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [15]:
from langchain_pinecone import PineconeVectorStore

vectorstore = PineconeVectorStore.from_documents(
    documents=documents,
    embedding=embeddings,
    index_name=index_name,
)

In [ ]:

list(dir(vectorstore)) # Check list of directories of vector store

In [ ]:
vectorstore

In [ ]:
vectorstore.similarity_search("fish")

In [ ]:
await vectorstore.asimilarity_search("parrot") # this is asynchronous

In [ ]:
vectorstore.similarity_search_with_score("human")

In [ ]:

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=2)  # select top result

retriever.batch(["friendly"])

In [22]:

from langchain_google_genai import ChatGoogleGenerativeAI



llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",
                             api_key = userdata.get('GOOGLE_API_KEY')
)


In [ ]:

llm.invoke("What's cat ?")

In [24]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context and some relative information from model.

{question}

Context:
{context}
"""

In [25]:
prompt = ChatPromptTemplate.from_messages([("human", message)])


In [26]:
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

In [27]:
from IPython.display import Markdown
response = rag_chain.invoke("tell about cats and its one bad habit and one good habit?")

print(response.content)


Based on the provided text, we only have information about cats being independent and enjoying their own space.  This could be interpreted as both a good habit (self-sufficiency) and a bad habit (potentially leading to aloofness or lack of interaction depending on the owner's preferences).  More information is needed to definitively state a good and bad habit.
